# Authenticate and Call M365 Copilot Retrieval API

This notebook demonstrates how to authenticate with an Entra app registration using delegated access (OAuth) and call the Microsoft 365 Copilot Retrieval API.

### 1. Install and Import Required Libraries

In [ ]:
!pip install msal requests

In [2]:
import msal
import requests
import json
import os
from dotenv import load_dotenv

# Find the .env file by searching up the directory tree
# This makes it independent of where the notebook is run from
current_dir = os.getcwd()
# Go up one level since the notebook is in the 'Copilot' folder
project_root = os.path.dirname(current_dir) 
dotenv_path = os.path.join(project_root, '.env')

# Check if the .env file exists at the constructed path
if not os.path.exists(dotenv_path):
    # If not found, assume the CWD is the project root
    dotenv_path = os.path.join(os.getcwd(), '.env')

load_dotenv(dotenv_path=dotenv_path)

True

### 2. Define Entra App and API Configuration

This notebook loads credentials from your root `.env` file. Make sure it contains the following variables:
- `CopilotAPIPythonClient_Id`
- `CopilotAPIPythonClient_Tenant`

**Note on Permissions:** This notebook uses **Delegated** permissions. In your Entra App Registration, grant the following permissions under **Delegated permissions** for Microsoft Graph:
- `Sites.Read.All`
- `ExternalItem.Read.All`
- `Files.Read.All`
- `User.Read`

1. In your App Registration, go to the **Authentication** tab.
2. Scroll down to **Advanced settings**.
3. Set the toggle for **Allow public client flows** to **Yes**.
4. Click **Save**.
5. Ensure you also have a **Mobile and desktop applications** platform configured with a redirect URI for `http://localhost`.

In [3]:
# Load Entra App Registration details from .env file
client_id = os.getenv("CopilotAPIPythonClient_Id")
tenant_id = os.getenv("CopilotAPIPythonClient_Tenant")
authority = f"https://login.microsoftonline.com/{tenant_id}"

# API details from the Swagger file
# Using the specific delegated permission scopes required by the API
scopes = [
    "https://graph.microsoft.com/Sites.Read.All",
    "https://graph.microsoft.com/ExternalItem.Read.All", 
    "https://graph.microsoft.com/Files.Read.All", 
    "https://graph.microsoft.com/User.Read"
]
api_endpoint = "https://graph.microsoft.com/v1.0/copilot/retrieval"

### 3. Acquire Access Token

This step uses the MSAL library to acquire an access token on behalf of the user. It will first try to get a token from the cache. If no token is cached, it will open a browser window for the user to sign in and consent.

In [4]:
app = msal.PublicClientApplication(
    client_id,
    authority=authority,
    # For more complex cache mechanisms, see MSAL documentation
    token_cache=msal.SerializableTokenCache()
)

# Try to get a token from the cache
accounts = app.get_accounts()
result = None
if accounts:
    # Assuming the user wants to use the first account found
    result = app.acquire_token_silent(scopes, account=accounts[0])

# If no cached token, or token is expired, acquire a new one interactively
if not result:
    print("No suitable token in cache. Initiating interactive login.")
    result = app.acquire_token_interactive(scopes=scopes)

if "access_token" in result:
    access_token = result["access_token"]
    print("Access token acquired successfully.")
else:
    print("Could not acquire access token.")
    print(result.get("error"))
    print(result.get("error_description"))
    access_token = None

No suitable token in cache. Initiating interactive login.
Access token acquired successfully.


### 4. Make an Authenticated API Call

Now we will use the acquired access token to make a `POST` request to the Copilot Retrieval API. The request body is based on the examples in the `copilot-retrieval-api.swagger.yaml` file.

In [5]:
if access_token:
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    # Example request body based on the swagger file
    retrieval_request = {
        "queryString": "What health benefit plans are available?",
        "dataSource": "sharePoint",
        "resourceMetadata": [
            "title",
            "author"
        ],
        "maximumNumberOfResults": 5
    }

    try:
        response = requests.post(api_endpoint, headers=headers, json=retrieval_request)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        
        api_response = response.json()
        print("API call successful.")

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
        print(f"Response body: {response.text}")
        api_response = None
    except Exception as err:
        print(f"An error occurred: {err}")
        api_response = None
else:
    print("Cannot make API call without an access token.")
    api_response = None

API call successful.


### 5. Print the API Response

Finally, let's print the JSON response from the API in a readable format.

In [6]:
if api_response:
    print(json.dumps(api_response, indent=2))

{
  "retrievalHits": [
    {
      "webUrl": "https://m365cpi89108028.sharepoint.com/Benefits/Northwind_Health_Plus_Benefits_Details.pdf",
      "extracts": [
        {
          "text": "<page_1>\r\n  \r\n \r\nContoso Electronics    \r\nNorthwind Health Plus Plan ![Contoso\r\nElectronics][image_/PGfuCkhdy5J4zpu65KvCw==] \r\n</page_1>\r\n<page_2>\r\n  \r\nThis document contains information generated using a language model \\(Azure OpenAI\\). The information contained in this document is only for demonstration purposes and does not reflect the opinions or beliefs of Microsoft. Microsoft makes no representations or warranties of any kind, express or implied, about the completeness, accuracy, reliability, suitability or availability with respect to the information contained in this document.    \r\nAll rights reserved to Microsoft \r\n</page_2>\r\n<page_3>\r\n\r\n## **Summary of Benefits **\r\n</page_3>\r\n<page_3>\r\n\r\n### **Northwind Health Plus **\r\n  \r\nNorthwind Health Plus is a 